In [6]:
from selenium import webdriver

In [7]:
driver=webdriver.Chrome(executable_path="chromedriver")

In [8]:
from lxml import html

In [9]:
import time
def find_page(link):
    root="https://www.avito.ru"
    listings=[]
    elements=link.xpath('//div[@class="iva-item-content-m2FiN"]/div[@class="iva-item-body-NPl6W"]')
    for element in elements:
        prop_name=element.xpath('./div[@class="iva-item-titleStep-2bjuh"]/a/@title')[0]
        url=element.xpath('./div[@class="iva-item-titleStep-2bjuh"]/a/@href')[0]
        price=element.xpath('./div[@class="iva-item-priceStep-2qRpg"]/span/span/span//text()')
        price_final=" ".join(price)
        #price=element.xpath('./div[@class="price-text-1HrJ_ text-text-1PdBw text-size-s-1PUdo"]')
        region=element.xpath('./div[@class="iva-item-developmentNameStep-1hr7p"]/div/span/span/text()')
        if len(region)>0:
            region=region[0]
        else:
            region=None
        listings.append({"prop_name":prop_name,"price":price_final,"region":region,"url":root+url})
    return listings

In [11]:
# Sometimes, the page will simply not load (bad connection, etc.). In that case, simply rerun this from the page that crashed
# So instead range(1,100+1), range(page_crashed,100+1)
all_listings=[]
for i in range(1,100+1):
    driver.get(f"https://www.avito.ru/rossiya/garazhi_i_mashinomesta?cd=1&p={i}")
    time.sleep(20)
    html_page=driver.page_source
    tree=html.fromstring(html_page)
    listings=find_page(tree)
    if len(listings)>0:
        all_listings.extend(listings)
        time.sleep(10)
    else:
        print(i)
        break

In [12]:
import pandas as pd
df_apartments=pd.DataFrame(all_listings)
df_apartments.to_csv('all_garage.csv',index=False,encoding='utf-8-sig')

In [14]:
driver.close()